# Customer Churn Analysis
Churn rate, when applied to a customer base, refers to the proportion of contractual customers or subscribers who leave a supplier during a given time period. It is a possible indicator of customer dissatisfaction, cheaper and/or better offers from the competition, more successful sales and/or marketing by the competition, or reasons having to do with the customer life cycle.

Churn is closely related to the concept of average customer life time. For example, an annual churn rate of 25 percent implies an average customer life of four years. An annual churn rate of 33 percent implies an average customer life of three years. The churn rate can be minimized by creating barriers which discourage customers to change suppliers (contractual binding periods, use of proprietary technology, value-added services, unique business models, etc.), or through retention activities such as loyalty programs. It is possible to overstate the churn rate, as when a consumer drops the service but then restarts it within the same year. Thus, a clear distinction needs to be made between "gross churn", the total number of absolute disconnections, and "net churn", the overall loss of subscribers or members. The difference between the two measures is the number of new subscribers or members that have joined during the same period. Suppliers may find that if they offer a loss-leader "introductory special", it can lead to a higher churn rate and subscriber abuse, as some subscribers will sign on, let the service lapse, then sign on again to take continuous advantage of current specials. https://en.wikipedia.org/wiki/Churn_rate

In [32]:
%%capture

# Get our favorite packages from PyPI
! pip install cufflinks

# Import pre-installed packages
import numpy as np
import pandas as pd

# Suppress unwatned warnings
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("requests").setLevel(logging.WARNING)

In [33]:
# Load our favorite visualization library
import os
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Sign into Plotly with masked, encrypted API key

myPlotlyKey = os.environ['SECRET_ENV_BRETTS_PLOTLY_KEY']
py.sign_in(username='bretto777',api_key=myPlotlyKey)


In [35]:
# Load some data
churnDF = pd.read_csv('https://s3-us-west-1.amazonaws.com/dsclouddata/home/jupyter/churn_train.csv', delimiter=',')
churnDF["Churn"] = churnDF["Churn"].replace(to_replace=False, value='Retain')
churnDF["Churn"] = churnDF["Churn"].replace(to_replace=True, value='Churn')
churnDFs = churnDF.sample(frac=0.07) # Sample for speedy viz
churnDF.head(10)

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn
0,ND,84,415,400-7253,no,yes,33,159.1,106,27.05,...,101,12.73,213.4,108,9.60,13.0,18,3.51,1,Retain
1,RI,117,408,370-5042,no,yes,13,207.6,65,35.29,...,77,12.98,232.8,95,10.48,9.7,3,2.62,1,Retain
2,VA,132,510,343-4696,no,no,0,81.1,86,13.79,...,72,20.84,237.0,115,10.67,10.3,2,2.78,0,Retain
3,OK,121,408,364-2495,no,yes,31,237.1,63,40.31,...,117,17.48,196.7,85,8.85,10.1,5,2.73,4,Retain
4,ME,205,510,413-4039,no,yes,24,175.8,139,29.89,...,98,13.18,180.7,64,8.13,7.8,5,2.11,2,Retain
5,MT,116,415,384-5907,no,yes,35,182.8,122,31.08,...,119,18.08,193.8,103,8.72,11.0,2,2.97,1,Retain
6,MN,113,408,417-5146,no,no,0,158.9,137,27.01,...,109,20.64,247.8,97,11.15,6.5,4,1.76,0,Retain
7,MO,74,415,421-2955,no,no,0,172.1,105,29.26,...,99,17.99,182.2,105,8.20,11.6,6,3.13,1,Retain
8,MI,112,510,420-1383,no,no,0,243.4,77,41.38,...,97,15.48,259.2,94,11.66,12.8,2,3.46,1,Retain
9,UT,68,415,403-8916,no,no,0,213.9,112,36.36,...,100,22.14,233.8,97,10.52,8.4,3,2.27,1,Churn


In [36]:
# separate the calls data for plotting


churnDFs = churnDFs[['Account Length','Day Calls','Eve Calls','CustServ Calls','Churn']]

# Create scatter plot matrix of call data
splom = ff.create_scatterplotmatrix(churnDFs, diag='histogram', index='Churn',  
                                  colormap= dict(
                                      Churn = '#9CBEF1',
                                      Retain = '#04367F'
                                      ),
                                  colormap_type='cat',
                                  height=560, width=650,
                                  size=4, marker=dict(symbol='circle'))
py.iplot(splom)

In [37]:
%%capture
import h2o
from h2o.automl import H2OAutoML
from __future__ import print_function
#h2o.connect(ip="35.225.239.147")
h2o.init(nthreads=2, max_mem_size="7G")

In [38]:
%%capture

# Split data into training and testing frames

from sklearn import cross_validation
from sklearn.model_selection import train_test_split

training, testing = train_test_split(churnDF, train_size=0.8, stratify=churnDF["Churn"], random_state=9)
train = h2o.H2OFrame(python_obj=training).drop("State")
test = h2o.H2OFrame(python_obj=testing).drop("State")

# Set predictor and response variables
y = "Churn"
x = train.columns
x.remove(y)

# Automatic Machine Learning

The Automatic Machine Learning (AutoML) function automates the supervised machine learning model training process. The current version of AutoML trains and cross-validates a Random Forest, an Extremely-Randomized Forest, a random grid of Gradient Boosting Machines (GBMs), a random grid of Deep Neural Nets, and a Stacked Ensemble of all the models.

In [39]:
%%capture
# Run AutoML until 11 models are built
autoModel = H2OAutoML(max_models = 11)
autoModel.train(x = x, y = y,
          training_frame = train,
          validation_frame = test, 
          leaderboard_frame = test)

## Leaderboard

In [40]:
leaders = autoModel.leaderboard
leaders

model_id,auc,logloss
GBM_grid_0_AutoML_20171208_144915_model_3,0.948764,0.136438
GBM_grid_0_AutoML_20171208_144915_model_0,0.947006,0.139041
GBM_grid_0_AutoML_20171208_144915_model_1,0.936963,0.136944
StackedEnsemble_AllModels_0_AutoML_20171208_144915,0.936091,0.130065
GBM_grid_0_AutoML_20171208_144915_model_2,0.934197,0.142858
GBM_grid_0_AutoML_20171208_144915_model_4,0.933039,0.194288
StackedEnsemble_BestOfFamily_0_AutoML_20171208_144915,0.932181,0.133007
DRF_0_AutoML_20171208_144915,0.918118,0.637457
XRT_0_AutoML_20171208_144915,0.916142,0.528189
GBM_grid_0_AutoML_20171208_144915_model_6,0.913131,0.221672


# Variable Importances
Below we plot variable importances as reported by the best performing algo in the ensemble.

In [41]:
importances = h2o.get_model(leaders[0, 0]).varimp(use_pandas=True)
importances = importances.loc[:,['variable','relative_importance']].groupby('variable').mean()
importances.sort_values(by="relative_importance", ascending=False).iplot(kind='bar', colors='#5AC4F2', theme='white')

# Super Model vs the Base models
This plot shows the ROC curves for the Super Model, the Best Base Model, and 9 next best models in the ensemble. 

In [42]:
Model0 = np.array(h2o.get_model(leaders[0,0]).roc(xval=True))
Model1 = np.array(h2o.get_model(leaders[1,0]).roc(xval=True))
Model2 = np.array(h2o.get_model(leaders[2,0]).roc(xval=True))
Model3 = np.array(h2o.get_model(leaders[3,0]).roc(xval=True))
Model4 = np.array(h2o.get_model(leaders[4,0]).roc(xval=True))
Model5 = np.array(h2o.get_model(leaders[5,0]).roc(xval=True))
Model6 = np.array(h2o.get_model(leaders[6,0]).roc(xval=True))
Model7 = np.array(h2o.get_model(leaders[7,0]).roc(xval=True))
Model8 = np.array(h2o.get_model(leaders[8,0]).roc(xval=True))
Model9 = np.array(h2o.get_model(leaders[9,0]).roc(xval=True))

layout = go.Layout(autosize=False, width=725, height=575,  xaxis=dict(title='False Positive Rate', titlefont=dict(family='Arial, sans-serif', size=15, color='grey')), 
                                                           yaxis=dict(title='True Positive Rate', titlefont=dict(family='Arial, sans-serif', size=15, color='grey')))

traceChanceLine = go.Scatter(x = [0,1], y = [0,1], mode = 'lines+markers', name = 'chance', line = dict(color = ('rgb(136, 140, 150)'), width = 4, dash = 'dash'))
Model0Trace = go.Scatter(x = Model0[0], y = Model0[1], mode = 'lines', name = 'Model 0', line = dict(color = ('rgb(26, 58, 126)'), width = 3))
Model1Trace = go.Scatter(x = Model1[0], y = Model1[1], mode = 'lines', name = 'Model 1', line = dict(color = ('rgb(156, 190, 241))'), width = 1))
Model2Trace = go.Scatter(x = Model2[0], y = Model2[1], mode = 'lines', name = 'Model 2', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model3Trace = go.Scatter(x = Model3[0], y = Model3[1], mode = 'lines', name = 'Model 3', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model4Trace = go.Scatter(x = Model4[0], y = Model4[1], mode = 'lines', name = 'Model 4', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model5Trace = go.Scatter(x = Model5[0], y = Model5[1], mode = 'lines', name = 'Model 5', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model6Trace = go.Scatter(x = Model6[0], y = Model6[1], mode = 'lines', name = 'Model 6', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model7Trace = go.Scatter(x = Model7[0], y = Model7[1], mode = 'lines', name = 'Model 7', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model8Trace = go.Scatter(x = Model8[0], y = Model8[1], mode = 'lines', name = 'Model 8', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model9Trace = go.Scatter(x = Model9[0], y = Model9[1], mode = 'lines', name = 'Model 9', line = dict(color = ('rgb(156, 190, 241)'), width = 1))

fig = go.Figure(data=[Model0Trace,Model1Trace,Model2Trace,Model3Trace,Model4Trace,Model5Trace,Model6Trace,Model8Trace,Model9Trace,traceChanceLine], layout=layout)

py.iplot(fig)

# Confusion Matrix

In [43]:
cm = h2o.get_model(leaders[1, 0]).confusion_matrix(xval=True)
cm = cm.table.as_data_frame()
cm
confusionMatrix = ff.create_table(cm)
confusionMatrix.layout.height=300
confusionMatrix.layout.width=800
confusionMatrix.layout.font.size=17
py.iplot(confusionMatrix)

# Business Impact Matrix

Weighting Predictions With a Dollar Value
-   Correctly predicting retain: `+$5`
-   Correctly predicting churn: `+$75`
-   Incorrectly predicting retain: `-$150`
-   Incorrectly predicting churn: `-$1.5`

    

In [44]:
CorrectPredictChurn = cm.loc[0,'Churn']
CorrectPredictChurnImpact = 75
cm1 = CorrectPredictChurn*CorrectPredictChurnImpact

IncorrectPredictChurn = cm.loc[1,'Churn']
IncorrectPredictChurnImpact = -5
cm2 = IncorrectPredictChurn*IncorrectPredictChurnImpact

IncorrectPredictRetain = cm.loc[0,'Retain']
IncorrectPredictRetainImpact = -150
cm3 = IncorrectPredictRetain*IncorrectPredictRetainImpact

CorrectPredictRetain = cm.loc[0,'Retain']
CorrectPredictRetainImpact = 5
cm4 = IncorrectPredictRetain*CorrectPredictRetainImpact


data_matrix = [['Business Impact', '($) Predicted Churn', '($) Predicted Retain', '($) Total'],
               ['($) Actual Churn', cm1, cm3, '' ],
               ['($) Actual Retain', cm2, cm4, ''],
               ['($) Total', cm1+cm2, cm3+cm4, cm1+cm2+cm3+cm4]]

impactMatrix = ff.create_table(data_matrix, height_constant=20, hoverinfo='weight')
impactMatrix.layout.height=300
impactMatrix.layout.width=800
impactMatrix.layout.font.size=17
py.iplot(impactMatrix)

In [45]:
print("Total customers evaluated: 2132")

Total customers evaluated: 2132


In [46]:
print("Total value created by the model: $" + str(cm1+cm2+cm3+cm4))

Total value created by the model: $5500.0


In [47]:
print("Total value per customer: $" +str(round(((cm1+cm2+cm3+cm4)/2132),3)))

Total value per customer: $2.58


In [29]:
# Save the best model

path = h2o.save_model(model=h2o.get_model(leaders[0, 0]), force=True)
os.rename(h2o.get_model(leaders[0, 0]).model_id, "AutoML-leader")    

In [31]:
LoadedEnsemble = h2o.load_model(path="AutoML-leader")
print(LoadedEnsemble)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_0_AutoML_20171207_233308_model_0


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0103609667706
RMSE: 0.101788834214
LogLoss: 0.0515738058879
Mean Per-Class Error: 0.0079859838887
AUC: 0.999133884138
Gini: 0.998267768277
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.765419767041: 


,Churn,Retain,Error,Rate
Churn,315.0,6.0,0.0187,(6.0/321.0)
Retain,4.0,1807.0,0.0022,(4.0/1811.0)
Total,319.0,1813.0,0.0047,(10.0/2132.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.7654198,0.9972406,230.0
max f2,0.6694659,0.9980161,246.0
max f0point5,0.8026400,0.9973448,224.0
max accuracy,0.7654198,0.9953096,230.0
max precision,0.9975043,1.0,0.0
max recall,0.6694659,1.0,246.0
max specificity,0.9975043,1.0,0.0
max absolute_mcc,0.7654198,0.9816223,230.0
max min_per_class_accuracy,0.8640426,0.9906542,215.0
max mean_per_class_accuracy,0.8306180,0.9920140,219.0


Gains/Lift Table: Avg response rate: 84.94 %



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.9516926,0.0072929,0.9601874,0.9344263,0.9624413,0.9553990,0.9460094
auc,0.908888,0.0086348,0.9161283,0.8992597,0.9263867,0.9108219,0.8918431
err,0.0483073,0.0072929,0.0398126,0.0655738,0.0375587,0.0446009,0.0539906
err_count,20.6,3.1176915,17.0,28.0,16.0,19.0,23.0
f0point5,0.9613627,0.0061094,0.9711643,0.9477652,0.9675704,0.9651985,0.9551148
f1,0.9720922,0.0042374,0.9767442,0.9617487,0.9784946,0.9739369,0.9695364
f2,0.9830886,0.0030610,0.9823886,0.9761509,0.9896683,0.982835,0.9844002
lift_top_group,1.1773996,0.0096749,1.179558,1.1960784,1.1671233,1.1866295,1.1576087
logloss,0.1816156,0.0181624,0.1617344,0.2276819,0.1539305,0.1849911,0.1797402
max_per_class_error,0.2690076,0.0467981,0.1846154,0.3285714,0.2459016,0.2238806,0.3620690


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2017-12-07 23:33:25,3.859 sec,0.0,0.3576223,0.4236842,0.5,1.0,0.1505629,0.3587174,0.4256264,0.5,1.0,0.1516854
,2017-12-07 23:33:25,3.896 sec,5.0,0.2643846,0.2586394,0.9307503,1.1772501,0.0323640,0.2722434,0.2693390,0.9173412,1.1788079,0.0411985
,2017-12-07 23:33:25,3.935 sec,10.0,0.2129434,0.1923477,0.9444705,1.1772501,0.0257974,0.2334570,0.2144748,0.9253945,1.1788079,0.0411985
,2017-12-07 23:33:25,3.990 sec,15.0,0.1816637,0.1541690,0.9588961,1.1772501,0.0229831,0.2139457,0.1869239,0.9180225,1.1788079,0.0393258
,2017-12-07 23:33:25,4.031 sec,20.0,0.1643581,0.1310328,0.9692232,1.1772501,0.0206379,0.2052419,0.1729929,0.9215654,1.1788079,0.0393258
,2017-12-07 23:33:25,4.070 sec,25.0,0.1533964,0.1146547,0.9774062,1.1772501,0.0201689,0.1977534,0.1601701,0.9320034,1.1788079,0.0393258
,2017-12-07 23:33:25,4.109 sec,30.0,0.1413787,0.0979454,0.9873317,1.1772501,0.0164165,0.1939355,0.1535015,0.9369362,1.1788079,0.0411985
,2017-12-07 23:33:25,4.146 sec,35.0,0.1293367,0.0822812,0.9947173,1.1772501,0.0150094,0.1892286,0.1478948,0.9411468,1.1788079,0.0374532
,2017-12-07 23:33:25,4.185 sec,40.0,0.1221310,0.0731330,0.9961279,1.1772501,0.0112570,0.1885743,0.1451096,0.9448805,1.1788079,0.0393258
,2017-12-07 23:33:25,4.223 sec,45.0,0.1159299,0.0656589,0.9968607,1.1772501,0.0093809,0.1870632,0.1421921,0.9469654,1.1788079,0.0355805


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
Day Mins,259.1791382,1.0,0.2387890
CustServ Calls,131.5142517,0.5074261,0.1211678
Day Charge,85.2524490,0.3289325,0.0785455
Intl Calls,79.7683640,0.3077731,0.0734928
Eve Charge,71.3329010,0.2752262,0.0657210
Int'l Plan,70.1283417,0.2705787,0.0646112
Eve Mins,69.7505569,0.2691210,0.0642631
Intl Charge,56.0279846,0.2161747,0.0516202
VMail Message,43.0298996,0.1660238,0.0396447
Intl Mins,41.8260612,0.1613790,0.0385355
